# Quantum Key Distribution
### A story of sending secrets over a quantum channel
![QKD in figures](../assets/qkd.png)
Although in modern network protocol sending secret values over the internet has become more secure than ever, there's still an issue that has not been solved and that is "how do we know if the secure data I sent from point A to B was not intercepted at a point C".

Well, the truth is in classical computing you can't really guarantee that because a bit will always be the same bit even if it gets intercepted a hundred time before getting to its destination. In other words, If I send `01101000 01101001` which is 'hi' in binary, over the network to someone on the other side of the world, there's a chance someone in the middle reads these bits and retransmits them without either the sender or the receiver ever knowing about it. This is not good because it means a secure channel could be compromised for a long time.

Enter, Quantum Key Distribution (QKD). Specifically we want to talk about BB84 QKD protocol, which promises secure transfer of secret messages and the ability to know if the message was intercepted. "But how?", you may ask. If you're into theory and wanna read the [original paper](https://web.archive.org/web/20200130165639/http://researcher.watson.ibm.com/researcher/files/us-bennetc/BB84highest.pdf) feel free. But let dive directly into code and go through how it works step by step.

In [1]:
# First let's import our libraries
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer.primitives import SamplerV2 as Sampler